In [1]:
import yfinance as yf
import keras.backend as K
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler ,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score, mean_squared_error,mean_absolute_percentage_error
import tensorflow as tf
from tensorflow import keras
from keras.layers import LSTM,Dropout, BatchNormalization, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from keras.models import Sequential
from keras.optimizer_v2.adam import Adam
from keras.optimizer_v2.rmsprop import RMSprop
import talib

In [2]:
# Get the data for the SPY ETF by specifying the stock ticker, start date, and end date
data = yf.download('0005.hk',"2017-09-28","2021-09-24")
# data.drop("Adj Close",axis=1,inplace=True)
data.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-09-16,40.599998,40.599998,39.599998,40.000000,40.000000,19553563
2021-09-17,39.700001,40.400002,39.099998,40.250000,40.250000,28266370
2021-09-20,39.900002,39.950001,38.950001,39.000000,39.000000,23796781
2021-09-21,38.349998,39.000000,38.299999,38.799999,38.799999,18117762
2021-09-23,39.299999,39.950001,39.200001,39.750000,39.750000,20518033


In [3]:
data['prediction'] = data['Close'].shift(-1)
data.dropna(inplace=True)
data

,Open,High,Low,Close,Adj Close,Volume,prediction
Date,,,,,,,
2017-09-28,76.400002,77.099998,76.050003,76.199997,64.386986,22520013,76.550003
2017-09-29,76.500000,76.900002,76.300003,76.550003,64.682732,16766252,77.449997
2017-10-03,76.800003,77.449997,76.699997,77.449997,65.443207,24369212,77.849998
2017-10-04,77.500000,77.949997,77.500000,77.849998,65.781197,21197563,77.349998
2017-10-06,77.599998,77.800003,77.199997,77.349998,65.358704,23715110,77.500000
...,...,...,...,...,...,...,...
2021-09-15,40.599998,40.599998,40.000000,40.099998,40.099998,14418368,40.000000
2021-09-16,40.599998,40.599998,39.599998,40.000000,40.000000,19553563,40.250000
2021-09-17,39.700001,40.400002,39.099998,40.250000,40.250000,28266370,39.000000


In [4]:
data_price=data["prediction"]
sc = MinMaxScaler(feature_range=(0,1))

def processData(data,data_price,lb,forecast_day):
    X,Y = [],[]
    for i in range(lb-1,len(data)-lb-1):
        X.append(data[i-(lb-1):i])
        Y.append(data_price[i:(i+forecast_day)])
    return np.array(X),np.array(Y)

lb=7
forecast=5
X,y = processData(data.dropna().values,data_price,lb,forecast)
X_train,X_test = X[:int(X.shape[0]*0.90)],X[int(X.shape[0]*0.90):]
y_train,y_test = y[:int(y.shape[0]*0.90)],y[int(y.shape[0]*0.90):]
# print(type(X_train))
print(X[0])

[[7.64000015e+01 7.70999985e+01 7.60500031e+01 7.61999969e+01
  6.43869858e+01 2.25200130e+07 7.65500031e+01]
 [7.65000000e+01 7.69000015e+01 7.63000031e+01 7.65500031e+01
  6.46827316e+01 1.67662520e+07 7.74499969e+01]
 [7.68000031e+01 7.74499969e+01 7.66999969e+01 7.74499969e+01
  6.54432068e+01 2.43692120e+07 7.78499985e+01]
 [7.75000000e+01 7.79499969e+01 7.75000000e+01 7.78499985e+01
  6.57811966e+01 2.11975630e+07 7.73499985e+01]
 [7.75999985e+01 7.78000031e+01 7.71999969e+01 7.73499985e+01
  6.53587036e+01 2.37151100e+07 7.75000000e+01]
 [7.73499985e+01 7.76999969e+01 7.70999985e+01 7.75000000e+01
  6.54854507e+01 1.65523540e+07 7.85999985e+01]]


In [5]:
data.head(15)

,Open,High,Low,Close,Adj Close,Volume,prediction
Date,,,,,,,
2017-09-28,76.400002,77.099998,76.050003,76.199997,64.386986,22520013,76.550003
2017-09-29,76.500000,76.900002,76.300003,76.550003,64.682732,16766252,77.449997
2017-10-03,76.800003,77.449997,76.699997,77.449997,65.443207,24369212,77.849998
2017-10-04,77.500000,77.949997,77.500000,77.849998,65.781197,21197563,77.349998
2017-10-06,77.599998,77.800003,77.199997,77.349998,65.358704,23715110,77.500000
2017-10-09,77.349998,77.699997,77.099998,77.500000,65.485451,16552354,78.599998
2017-10-10,77.500000,79.650002,77.500000,78.599998,66.414932,54726340,78.099998
2017-10-11,79.050003,79.199997,78.099998,78.099998,65.992439,33386628,77.300003
2017-10-12,77.699997,77.750000,77.300003,77.300003,65.975250,14127044,77.199997


In [6]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(100, return_sequences=True, input_shape= (X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(100, return_sequences=False))
model.add(Dropout(0.2))
# model.add(Dense(32))
model.add(Dense(1))
model.compile(
  loss="mean_squared_error",
  optimizer='Adam'
)

In [7]:
#Fit model with history to check for overfitting
# es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=40, min_delta=0.0001)
history = model.fit(X_train,y_train,epochs=300,batch_size=20,validation_data=(X_test,y_test),shuffle=False)

Epoch 1/300
44/44 [==============================] - 10s 20ms/step - loss: 4409.2638 - val_loss: 904.9272
Epoch 2/300
44/44 [==============================] - 0s 6ms/step - loss: 2957.9253 - val_loss: 615.1413
Epoch 3/300
44/44 [==============================] - 0s 6ms/step - loss: 2462.7084 - val_loss: 427.3309
Epoch 4/300
44/44 [==============================] - 0s 6ms/step - loss: 2088.8226 - val_loss: 287.9947
Epoch 5/300
44/44 [==============================] - 0s 6ms/step - loss: 1779.9126 - val_loss: 185.2335
Epoch 6/300
44/44 [==============================] - 0s 6ms/step - loss: 1511.1291 - val_loss: 111.7632
Epoch 7/300
44/44 [==============================] - 0s 6ms/step - loss: 1308.3190 - val_loss: 61.8778
Epoch 8/300
44/44 [==============================] - 0s 6ms/step - loss: 1118.2225 - val_loss: 30.6932
Epoch 9/300
44/44 [==============================] - 0s 6ms/step - loss: 955.7366 - val_loss: 14.0353
Epoch 10/300
44/44 [==============================] - 0s 6ms/step 

Epoch 80/300
44/44 [==============================] - 0s 6ms/step - loss: 266.4981 - val_loss: 181.7872
Epoch 81/300
44/44 [==============================] - 0s 6ms/step - loss: 262.0310 - val_loss: 180.9335
Epoch 82/300
44/44 [==============================] - 0s 5ms/step - loss: 273.0454 - val_loss: 181.7556
Epoch 83/300
44/44 [==============================] - 0s 5ms/step - loss: 265.8360 - val_loss: 181.2229
Epoch 84/300
44/44 [==============================] - 0s 6ms/step - loss: 267.3760 - val_loss: 181.1848
Epoch 85/300
44/44 [==============================] - 0s 6ms/step - loss: 262.8326 - val_loss: 182.0091
Epoch 86/300
44/44 [==============================] - 0s 6ms/step - loss: 264.1753 - val_loss: 182.2416
Epoch 87/300
44/44 [==============================] - 0s 6ms/step - loss: 271.1138 - val_loss: 182.3170
Epoch 88/300
44/44 [==============================] - 0s 6ms/step - loss: 254.9367 - val_loss: 181.4869
Epoch 89/300
44/44 [==============================] - 0s 6ms/ste

44/44 [==============================] - 0s 6ms/step - loss: 268.3546 - val_loss: 181.5088
Epoch 159/300
44/44 [==============================] - 0s 6ms/step - loss: 265.1853 - val_loss: 181.5855
Epoch 160/300
44/44 [==============================] - 0s 6ms/step - loss: 259.0561 - val_loss: 180.5099
Epoch 161/300
44/44 [==============================] - 0s 6ms/step - loss: 271.1607 - val_loss: 180.9589
Epoch 162/300
44/44 [==============================] - 0s 6ms/step - loss: 271.2942 - val_loss: 181.3082
Epoch 163/300
44/44 [==============================] - 0s 6ms/step - loss: 271.8217 - val_loss: 181.5863
Epoch 164/300
44/44 [==============================] - 0s 6ms/step - loss: 273.9620 - val_loss: 181.8275
Epoch 165/300
44/44 [==============================] - 0s 6ms/step - loss: 265.0677 - val_loss: 181.4296
Epoch 166/300
44/44 [==============================] - 0s 6ms/step - loss: 270.8707 - val_loss: 181.1902
Epoch 167/300
44/44 [==============================] - 0s 6ms/step - 

44/44 [==============================] - 0s 6ms/step - loss: 264.9503 - val_loss: 182.4796
Epoch 237/300
44/44 [==============================] - 0s 6ms/step - loss: 264.7821 - val_loss: 182.1240
Epoch 238/300
44/44 [==============================] - 0s 6ms/step - loss: 260.1360 - val_loss: 182.0965
Epoch 239/300
44/44 [==============================] - 0s 6ms/step - loss: 266.9004 - val_loss: 180.9645
Epoch 240/300
44/44 [==============================] - 0s 6ms/step - loss: 265.4215 - val_loss: 180.6159
Epoch 241/300
44/44 [==============================] - 0s 6ms/step - loss: 275.1278 - val_loss: 180.6530
Epoch 242/300
44/44 [==============================] - 0s 6ms/step - loss: 258.9608 - val_loss: 179.8508
Epoch 243/300
44/44 [==============================] - 0s 6ms/step - loss: 267.7603 - val_loss: 179.9034
Epoch 244/300
44/44 [==============================] - 0s 6ms/step - loss: 265.6816 - val_loss: 179.0644
Epoch 245/300
44/44 [==============================] - 0s 6ms/step - 

In [8]:
y_pred = model.predict(X_test)

In [9]:
print(X_test.shape)
y_pred.shape

(97, 6, 7)


(97, 1)

In [10]:
# Get the data for the SPY ETF by specifying the stock ticker, start date, and end date
df = yf.download('0005.hk',"2021-09-22","2021-09-30")
# data.drop("Adj Close",axis=1,inplace=True)
df.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-09-23,39.299999,39.950001,39.200001,39.750000,39.750000,20518033
2021-09-24,40.200001,40.799999,39.700001,39.700001,39.700001,19052618
2021-09-27,40.000000,41.099998,40.000000,40.299999,40.299999,20332913
2021-09-28,41.250000,41.250000,40.599998,40.849998,40.849998,29545281
2021-09-29,40.500000,41.599998,40.299999,41.250000,41.250000,30287737
